# Quantum `conda` environment

Let's create a new `conda` environment called "q-env" and running Python 3.10:

```bash
conda create -n q-env python=3.11 -y
```

After the activation

```bash
conda activate q-env
```

start by upgrading `pip`, `ipykernel`, and `numpy` via:

```bash
pip install --upgrade pip ipykernel numpy==1.26.4
```

Continue by installing PyTorch 2.3 based on CUDA 12.1:

```bash
pip install torch==2.3.* --index-url https://download.pytorch.org/whl/cu121
```

Test that the installation succeeds and that PyTorch correctly accesses the GPU by executing what follows:

In [1]:
import torch

print(f"[DEBUG] PyTorch version {torch.__version__}")

torch.cuda.init()

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
    rnd = torch.randn(size=(100, 1)).to(device)
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

[DEBUG] PyTorch version 2.3.1+cu121
[STATUS] GPU available


Continue by installing Jax based on CUDA 12.1 (via `pip` channel):

pip install --upgrade "jax[cuda12_pip]==0.4.28" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
Test that the installation succeeds and that Jax correctly accesses the GPU by executing what follows:

In [2]:
import jax

print(f"[DEBUG] Jax version {jax.__version__}")

def jax_has_gpu():
    try:
        _ = jax.device_put(jax.numpy.ones(1), device=jax.devices('gpu')[0])
        return True
    except:
        return False

if jax_has_gpu():
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

[DEBUG] Jax version 0.4.28
[STATUS] GPU available


2024-06-17 11:21:12.159879: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Continue by installing Pennylane Lightning with GPU-acceleration:

```bash
pip install pennylane pennylane-lightning-gpu
```

To install Lightning with NVIDIA CUDA support, the following packages need to be installed via `pip`:

```bash
pip install nvidia-cusparse-cu12 nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 custatevec_cu12
```

Test that the installation succeeds and that Pennylane Lightning correctly accesses the GPU by executing what follows:

In [3]:
import pennylane as qml

print(f"[DEBUG] PennyLane version {qml.__version__}")

dev = qml.device("lightning.gpu", wires=2)

@qml.qnode(dev)
def circuit():
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0,1])
  return qml.expval(qml.PauliZ(0))

circuit()
# tensor(0., requires_grad=True)

[DEBUG] PennyLane version 0.36.0


tensor(0., requires_grad=True)

Continue by installing Qiskit Aer with GPU-acceleration (based on CUDA 12):

```bash
pip install qiskit qiskit-aer-gpu
```

Test that the installation succeeds and that Qiskit Aer correctly accesses the GPU by executing what follows:

In [4]:
import qiskit
from qiskit_aer import AerSimulator

print(f"[DEBUG] Qiskit version {qiskit.__version__}")

# Generate 3-qubit GHZ state
circ = qiskit.QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.measure_all()

# Construct an ideal simulator
aersim = AerSimulator()

# Perform an ideal simulation
result_ideal = aersim.run(circ).result()
counts_ideal = result_ideal.get_counts(0)
print('[DEBUG] Counts(ideal):', counts_ideal)
# Counts(ideal): {'000': 493, '111': 531}

[DEBUG] Qiskit version 1.1.0
[DEBUG] Counts(ideal): {'000': 514, '111': 510}


Finally, continue by installing Qibo, QiboJIT, and CuPy via `conda`:

```bash
conda install qibo qibojit -y
conda install -c conda-forge cupy cuda-version=12.1 -y
```

In [5]:
import cupy as cp

print(f"[DEBUG] CuPy version {cp.__version__}")

n_cuda_devices = cp.cuda.runtime.getDeviceCount()

if n_cuda_devices > 0:
    x = cp.array([1, 2, 3])
    x.device
    print("[STATUS] GPU available")
else:
    print("[STATUS] GPU not available")

[DEBUG] CuPy version 13.2.0
[STATUS] GPU available
